# Preprocessing Movie Data

In [27]:
# Import Libraries 
import pandas as pd
import numpy as np
import os

## Import Dataframes

In [3]:
# Save all url's for the datasets
basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'
akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'

In [4]:
# Convert data sets into dataframes
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [5]:
# Replace \N with NaN in all the dataframes
basics.replace({'\\N':np.nan}, inplace=True)
ratings.replace({'\\N':np.nan}, inplace=True)
akas.replace({'\\N':np.nan}, inplace=True)

In [6]:
# check for duplicate rows in dataframes
print(f'Duplicate rows in basics dataframe:', basics.duplicated().sum())
print(f'Duplicate rows in ratings dataframe:', ratings.duplicated().sum())
print(f'Duplicate rows in akas dataframe:', akas.duplicated().sum())

Duplicate rows in basics dataframe: 0
Duplicate rows in ratings dataframe: 0
Duplicate rows in akas dataframe: 0


## Cleaning Dataframes

### Cleaning AKAs Dataframe

In [7]:
# Display info for AKAs dataframe
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35165559 entries, 0 to 35165558
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.1+ GB


In [8]:
# Check value_counts for region column
akas['region'].value_counts()

DE    4205742
FR    4201899
JP    4201560
IN    4140891
ES    4123662
       ...   
CC          1
TV          1
NU          1
PW          1
NR          1
Name: region, Length: 247, dtype: int64

In [9]:
# Keep only movies in US region
akas = akas.loc[akas['region'] == 'US']
akas['region'].value_counts()

US    1419540
Name: region, dtype: int64

In [10]:
# Check info of akas dataframe
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1419540 entries, 5 to 35165303
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1419540 non-null  object
 1   ordering         1419540 non-null  int64 
 2   title            1419540 non-null  object
 3   region           1419540 non-null  object
 4   language         3844 non-null     object
 5   types            975140 non-null   object
 6   attributes       46137 non-null    object
 7   isOriginalTitle  1418195 non-null  object
dtypes: int64(1), object(7)
memory usage: 97.5+ MB


### Cleaning Title Basics Dataframe

In [11]:
# Display info for title basics dataframe
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9669713 entries, 0 to 9669712
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 664.0+ MB


In [12]:
# Eliminate movies that are null for runtimeMinutes
print(f'NaN count in runtimeMinutes column:', basics['runtimeMinutes'].isna().sum())
basics.dropna(subset=['runtimeMinutes'], inplace=True)
print(f'NaN count after eliminating null rows:',basics['runtimeMinutes'].isna().sum())

NaN count in runtimeMinutes column: 6834026
NaN count after eliminating null rows: 0


In [13]:
# Eliminate movies that are null for genre
print(f'NaN count in genre column:', basics['genres'].isna().sum())
basics.dropna(subset=['genres'], inplace=True)
print(f'NaN count after eliminating null rows:',basics['genres'].isna().sum())

NaN count in genre column: 75976
NaN count after eliminating null rows: 0


In [14]:
# Check value_counts for the titleType column
basics['titleType'].value_counts()

tvEpisode       1384066
short            593724
movie            378414
video            179264
tvMovie           91013
tvSeries          89533
tvSpecial         17775
tvMiniSeries      16921
tvShort            8683
videoGame           318
Name: titleType, dtype: int64

In [15]:
# Keep only rows in which titleType == movie
basics = basics.loc[basics['titleType'] == 'movie']
basics['titleType'].value_counts()

movie    378414
Name: titleType, dtype: int64

In [16]:
# Remove movies that are null for the startYear
# we will do this because we only want to keep movies with a startYear between
# 2000 - 2021, and null values are not between those years
basics.dropna(subset = ['startYear'], inplace=True)
basics['startYear'].isna().sum()

0

In [17]:
# convert start year column to type integer
basics['startYear'] = basics['startYear'].astype(int)
basics.dtypes

tconst            object
titleType         object
primaryTitle      object
originalTitle     object
isAdult           object
startYear          int32
endYear           object
runtimeMinutes    object
genres            object
dtype: object

In [18]:
# Keep rows with startYear between 2000 - 2022
basics = basics.loc[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2021)]

In [19]:
# check that startYear column only kept data between 2000 - 2021
basics['startYear'].describe()

count    209638.000000
mean       2012.838588
std           5.611237
min        2000.000000
25%        2009.000000
50%        2014.000000
75%        2017.000000
max        2021.000000
Name: startYear, dtype: float64

In [20]:
# Eliminate movies that include 'Documentary' in the genre 
is_documentary = basics['genres'].str.contains('documentary', case=False)
basics = basics[~is_documentary]

In [21]:
# Keep only movies made in the US region (as defined by the akas dataframe)
keepers = basics['tconst'].isin(akas['titleId'])
basics = basics[keepers]

In [22]:
# Check info of basics dataframe
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81112 entries, 34803 to 9669479
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          81112 non-null  object
 1   titleType       81112 non-null  object
 2   primaryTitle    81112 non-null  object
 3   originalTitle   81112 non-null  object
 4   isAdult         81112 non-null  object
 5   startYear       81112 non-null  int32 
 6   endYear         0 non-null      object
 7   runtimeMinutes  81112 non-null  object
 8   genres          81112 non-null  object
dtypes: int32(1), object(8)
memory usage: 5.9+ MB


### Cleaning Title Ratings Dataframe

In [23]:
# check info for ratings dataframe
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1286868 entries, 0 to 1286867
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1286868 non-null  object 
 1   averageRating  1286868 non-null  float64
 2   numVotes       1286868 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 29.5+ MB


In [24]:
# Keep only movies made in the US region (as defined by the akas dataframe)
keepers = ratings['tconst'].isin(akas['titleId'])
ratings = ratings[keepers]

In [25]:
# check info of ratings dataframe
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 491345 entries, 0 to 1286846
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         491345 non-null  object 
 1   averageRating  491345 non-null  float64
 2   numVotes       491345 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.0+ MB


## Saving Dataframes in the GitHub Repository

In [28]:
# Create 'Data/' file in GitHub Repository
os.makedirs('Data/', exist_ok=True)
# confirm the folder is created
os.listdir("Data/")

['blue_long_2-9665a76b1ae401a510ec1e0ca40ddcb3b0cfe45f1d51b77a308fea0845885648.svg',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'test_json.json',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

In [29]:
# save dataframes to files
basics.to_csv("Data/title_basics.csv.gz", compression='gzip', index=False)
ratings.to_csv("Data/title_ratings.csv.gz", compression='gzip', index=False)
akas.to_csv("Data/title_akas.csv.gz", compression='gzip', index=False)

In [30]:
# check basics dataframe was saved correctly
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81112 entries, 0 to 81111
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          81112 non-null  object 
 1   titleType       81112 non-null  object 
 2   primaryTitle    81112 non-null  object 
 3   originalTitle   81112 non-null  object 
 4   isAdult         81112 non-null  int64  
 5   startYear       81112 non-null  int64  
 6   endYear         0 non-null      float64
 7   runtimeMinutes  81112 non-null  int64  
 8   genres          81112 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 5.6+ MB


In [31]:
# check ratings dataframe was saved correctly
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory=False)
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491345 entries, 0 to 491344
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         491345 non-null  object 
 1   averageRating  491345 non-null  float64
 2   numVotes       491345 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 11.2+ MB


In [32]:
# check akas dataframe was saved correctly
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory=False)
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1419540 entries, 0 to 1419539
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1419540 non-null  object 
 1   ordering         1419540 non-null  int64  
 2   title            1419540 non-null  object 
 3   region           1419540 non-null  object 
 4   language         3844 non-null     object 
 5   types            975140 non-null   object 
 6   attributes       46137 non-null    object 
 7   isOriginalTitle  1418195 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 86.6+ MB
